In [1]:
%matplotlib inline
from ipywidgets import interact, BoundedFloatText, FloatSlider, Layout, SliderStyle
import numpy as np
import matplotlib.pyplot as plt
from ctrl_sim import *
import time

class Visualizer:
    def __init__(self):
        self.m_input = BoundedFloatText(value=1.0, min=0, max=10.0, step=0.1, description='質量:', disabled=False)
        self.c_input = BoundedFloatText(value=0.5, min=0, max=10.0, step=0.1, description='ダンパ:', disabled=False)
        self.k_input = BoundedFloatText(value=1.0, min=0, max=10.0, step=0.1, description='ばね:', disabled=False)
        self.m = 1.0
        self.c = 0.5
        self.k = 1.0
        
        # 各ウィジェットに対してset_model_paramをイベントハンドラとして登録
        self.m_input.observe(self.set_model_param, names='value')
        self.c_input.observe(self.set_model_param, names='value')
        self.k_input.observe(self.set_model_param, names='value')
        
        # ウィジェットを表示
        display(self.m_input, self.c_input, self.k_input)
        
        # スライダーを作成
        interact(self.on_slide, kp=FloatSlider(min=0.0, max=20.0, step=0.001, value=10.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')), 
                 ki=FloatSlider(min=0.0, max=20.0, step=0.001, value=2.2, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')), 
                 kd=FloatSlider(min=0.0, max=20.0, step=0.001, value=4.13, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')))

    # 入力変更時に呼ばれる関数。値を更新して出力
    def set_model_param(self, change):
        # 各入力ボックスの値を対応する属性に更新
        self.m = self.m_input.value
        self.c = self.c_input.value
        self.k = self.k_input.value
    
    # シミュレーション実行メソッド
    def simulate(self, param):
        m, c, k, kp, ki, kd = param
        sys = Damped_Mass_Spring_System(m, k, c)
        ctrl = PID_Controller(2, 1)
        dms = Simulator(sys, ctrl)
    
        maxT = 30
        dt = 0.01
        dms.set_aim([0, 0], [10, 0], [10])
        dms.ctrl.set_param('kp', [[kp, 0.0]])
        dms.ctrl.set_param('ki', [[ki, 0.0]])
        dms.ctrl.set_param('kd', [[kd, 0.0]])
        t_start = time.perf_counter()
        self.is_attained = dms.execute_until_stationary(maxT=maxT, dt=dt, threshold=0.1)
        t_control = time.perf_counter()
        self.plot_result(dms)
        t_plot = time.perf_counter()
        print(f'control calc time: {t_control - t_start:.3f}sec; plot time: {t_plot - t_control:.3f}sec')

    def plot_result(self, dms):
        plt.figure(figsize=(12, 4))

        m = dms.sys.param.get('m')
        k = dms.sys.param.get('k')
        c = dms.sys.param.get('c')
        kp = dms.ctrl.get_param('kp')[0,0]
        ki = dms.ctrl.get_param('ki')[0,0]
        kd = dms.ctrl.get_param('kd')[0,0]

        plt.subplot(1, 2, 1)
        title_str = 'Damped-Mass-Spring Model PID Simulation: '
        if self.is_attained:
            title_str += 'Attained in %.3f sec\n' % (dms.sys.t)
        else:
            title_str += 'not attained\n'
        title_str += '(m, k, c) = (%.1f, %.1f, %.1f); ' % (m, k, c)
        title_str += '(kp, ki, kd) = (%.3f, %.3f, %.3f)' % (kp, ki, kd)
        plt.title(title_str)
        
        time = np.array([d[0] for d in dms.history])
        x_array = np.array([d[1] for d in dms.history])
        u_array = np.array([d[2] for d in dms.history])
        
        plt.plot(time, x_array[:, 0], label='Position')
        plt.plot(time, x_array[:, 1], label='Velocity')
        plt.plot(time, u_array[:, 0], label='Control Input')
        plt.xlabel('Time (s)')
        plt.ylabel('State')
        plt.legend()
        plt.grid(True)

        plt.subplot(1, 2, 2)
        charpol = np.poly1d([m, c + kd, k + kp, ki])
        min_rt = np.inf
        max_rt = -np.inf
        hz = 0.0
        wav_amp = 0.0
        for z in charpol.roots:
            if np.abs(z.imag) >= 1.0e-8:
                hz = np.abs(z.imag / (2*np.pi))
                wav_amp = z.real
            min_rt = min(min_rt, z.real)
            max_rt = max(max_rt, z.real)
            
        rt_dist = max_rt - min_rt
        if rt_dist == 0:
            rt_dist = 1.0
        la_min = min_rt - rt_dist
        la_max = max_rt + rt_dist
        la = np.linspace(la_min, la_max, 100)
        charval = list(map(charpol, la))

        chartitle = '$%.1f\lambda^3 + %.1f\lambda^2 + %.1f\lambda + %.1f$: ' % tuple(charpol.c)
        if max_rt <= 0:
            chartitle += 'Converges (%.3f)\n' % max_rt
        else:
            chartitle += 'Diverges (%.3f)\n' % max_rt
        if hz == 0.0:
            chartitle += 'no vibration'
        else:
            chartitle += 'vibration: %.3f Hz (%.3f) ' % (hz, wav_amp)

        plt.title(chartitle)
        plt.plot(la, charval)
        plt.xlabel('Lambda')
        plt.ylabel('Char pol value')
        plt.grid(True)

    # スライダー変更時に呼ばれる関数
    def on_slide(self, kp, ki, kd):
        self.simulate((self.m, self.c, self.k, kp, ki, kd))

vis = Visualizer()


BoundedFloatText(value=1.0, description='質量:', max=10.0, step=0.1)

BoundedFloatText(value=0.5, description='ダンパ:', max=10.0, step=0.1)

BoundedFloatText(value=1.0, description='ばね:', max=10.0, step=0.1)

interactive(children=(FloatSlider(value=10.0, description='kp', layout=Layout(width='100%'), max=20.0, step=0.…

In [3]:
%matplotlib inline
from ipywidgets import interact, BoundedFloatText, FloatSlider, BoundedIntText, Button, Layout, SliderStyle
from IPython.display import display, Math
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from ctrl_sim import *

class MPC_param:
   def __init__(self, param_horizon, param_cost):
    self.K, self.step_time = param_horizon
    self.Q, self.R, self.Qf = param_cost

class VisualizerMPC:
    def __init__(self):
        self.m_input = BoundedFloatText(value=1.0, min=0, max=10.0, step=0.1, description='質量:', disabled=False)
        self.c_input = BoundedFloatText(value=0.5, min=0, max=10.0, step=0.1, description='ダンパ:', disabled=False)
        self.k_input = BoundedFloatText(value=1.0, min=0, max=10.0, step=0.1, description='ばね:', disabled=False)
        self.horizon_input = BoundedIntText(value=30, min=1, max=1000, step=1, description='ホライズン長:', disabled=False)
        self.step_time_input = BoundedFloatText(value=0.01, min=0.00001, max=100.0, description='ステップ時間:', disabled=False)
        
        self.m = 1.0
        self.c = 0.5
        self.k = 1.0
        self.horizon = 50
        self.step_time = 0.01
        
        # 各ウィジェットに対してset_model_paramをイベントハンドラとして登録
        self.m_input.observe(self.set_model_param, names='value')
        self.c_input.observe(self.set_model_param, names='value')
        self.k_input.observe(self.set_model_param, names='value')
        self.horizon_input.observe(self.set_model_param, names='value')
        self.step_time_input.observe(self.set_model_param, names='value')
        
        # ウィジェットを表示
        display(self.m_input, self.c_input, self.k_input, self.horizon_input, self.step_time_input)
        
        # スライダーを作成
        interact(self.on_cost_slide,
                 # stage cost = Qxx*x^2 + Qxv * x*v + Qvv*v^2 + Quu*u^2
                 Qxx=FloatSlider(min=0.0, max=100.0, step=0.1, value=10.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue', continuous_update=False)),
                 Qxv=FloatSlider(min=0.0, max=100.0, step=0.1, value=0.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue', continuous_update=False)),
                 Qvv=FloatSlider(min=0.0, max=100.0, step=0.1, value=0.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue', continuous_update=False)),
                 Quu=FloatSlider(min=0.0, max=100.0, step=0.1, value=0.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue', continuous_update=False)),
                 
                 # terminal cost = Qfxx*x^2 + Qfxv * x*v + Qfvv*v^2
                 Qfxx=FloatSlider(min=0.0, max=100.0, step=0.1, value=10.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue', continuous_update=False)),
                 Qfxv=FloatSlider(min=0.0, max=100.0, step=0.1, value=0.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue', continuous_update=False)),
                 Qfvv=FloatSlider(min=0.0, max=100.0, step=0.1, value=0.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue', continuous_update=False))
                )

    # 入力変更時に呼ばれる関数。値を更新して出力
    def set_model_param(self, change):
        # 各入力ボックスの値を対応する属性に更新
        self.m = self.m_input.value
        self.c = self.c_input.value
        self.k = self.k_input.value
        self.horizon = self.horizon_input.value
        self.step_time = self.step_time_input.value
    
    # シミュレーション実行メソッド
    def simulate(self, param):
        sys = Damped_Mass_Spring_System(self.m, self.k, self.c)
        ctrl = MPC_Controller(2, 1)
        ctrl.set_model(sys.f)
        dms = Simulator(sys, ctrl)
        self.dms = dms
        self.param = param
        
        maxT = 10
        dt = 0.01

        dms.ctrl.set_horizon(param.K, param.K*param.step_time)
        dms.ctrl.set_constraint(np.array([-0.01, -0.01]), np.array([10.01, 10.01]), np.array([-np.inf]), np.array([np.inf]))
        dms.ctrl.set_cost(param.Q, param.R, param.Qf)
        dms.set_aim([0, 0], [10, 0], [10])
        dms.ctrl.set_solver()
        dms.ctrl.opt_history = []
        dms.ctrl.set_record(True)
        
        t_start = time.perf_counter()
        self.is_attained = dms.execute_until_stationary(maxT=maxT, dt=dt, threshold=0.1)
        t_control = time.perf_counter()
        self.plot_result(dms, param)
        t_plot = time.perf_counter()
        print(f'control calc time: {t_control - t_start:.3f}sec; plot time: {t_plot - t_control:.3f}sec')
        interact(self.on_timeline_slide,
                 t=FloatSlider(min=0.0, max=dms.sys.t - dt, step=dt, value=dms.sys.t - dt, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue'))
                )

    def plot_result(self, dms, param, t=-1):
        plt.figure(figsize=(12, 4))
        time = np.array([d[0] for d in dms.history])
        x_array = np.array([d[1] for d in dms.history])
        u_array = np.array([d[2] for d in dms.history])

        m = dms.sys.param.get('m')
        k = dms.sys.param.get('k')
        c = dms.sys.param.get('c')
        
        plt.subplot(1, 1, 1)
        title_str = 'Damped-Mass-Spring Model MPC Simulation: '
        if self.is_attained:
            title_str += 'Attained in %.3f sec\n' % (dms.sys.t)
        else:
            title_str += 'not attained\n'
        title_str += '(m, k, c) = (%.1f, %.1f, %.1f); ' % (m, k, c)
        plt.title(title_str)

        if t >= 0:
            cur_step = int(t / 0.01 + 0.5)
        else:
            cur_step = len(time)-1

        color_list = ['tab:blue', 'tab:orange', 'tab:green']        
        plt.plot(time[:cur_step+1], x_array[:, 0][:cur_step+1], label='Position', color=color_list[0])
        plt.plot(time[:cur_step+1], x_array[:, 1][:cur_step+1], label='Velocity', color=color_list[1])
        plt.plot(time[:cur_step+1], u_array[:cur_step+1, 0], label='Control Input', color=color_list[2])

        if t >= 0:
            hist = self.dms.ctrl.opt_history[cur_step]
            plt.plot(list(t + s*self.dms.ctrl.param['dt'] for s in np.arange(self.dms.ctrl.param['K']+1)), hist[0].full()[0], linestyle='--', color=color_list[0])
            plt.plot(list(t + s*self.dms.ctrl.param['dt'] for s in np.arange(self.dms.ctrl.param['K']+1)), hist[0].full()[1], linestyle='--', color=color_list[1])
            plt.plot(list(t + s*self.dms.ctrl.param['dt'] for s in np.arange(self.dms.ctrl.param['K'])), hist[1].full()[0], linestyle='--', color=color_list[2])
            plt.scatter(t, x_array[:, 0][cur_step], color=color_list[0])
            plt.scatter(t, x_array[:, 1][cur_step], color=color_list[1])
        else:
            t, state = dms.sys.observe()
            state = state.full()
            plt.scatter(dms.sys.t, state[0], color=color_list[0])
            plt.scatter(dms.sys.t, state[1], color=color_list[1])
        
        plt.xlabel('Time (s)')
        plt.ylabel('State')
        plt.legend()
        plt.grid(True)

    def is_positive_semidefinite_sym2x2(self, Q):
        return Q[0][1] == Q[1][0] and Q[0][0] >= 0 and Q[0][0]*Q[1][1] >= Q[0][1]*Q[0][1]

    # スライダー変更時に呼ばれる関数
    def on_cost_slide(self, Qxx, Qxv, Qvv, Quu, Qfxx, Qfxv, Qfvv):
        Qx = np.array([[Qxx, 0.5*Qxv], [0.5*Qxv, Qvv]])
        Qu = np.array([Quu])
        Qf = np.array([[Qfxx, 0.5*Qfxv], [0.5*Qfxv, Qfvv]])
        display(Math(rf"""
            \begin{{equation}}
                Q = {sp.latex(sp.Matrix(Qx))}, \quad R = {sp.latex(sp.Matrix(Qu))}, \quad Q_f = {sp.latex(sp.Matrix(Qf))}
            \end{{equation}}
            """))
        
        if self.is_positive_semidefinite_sym2x2(Qx) and Quu >= 0 and self.is_positive_semidefinite_sym2x2(Qf):
            self.simulate(MPC_param((self.horizon, self.step_time), (Qx, Qu, Qf)))
        else:
            print("The cost function is not positive semi-definite!!!")

    def on_timeline_slide(self, t):
        self.plot_result(self.dms, self.param, t=t)

vis = VisualizerMPC()


BoundedFloatText(value=1.0, description='質量:', max=10.0, step=0.1)

BoundedFloatText(value=0.5, description='ダンパ:', max=10.0, step=0.1)

BoundedFloatText(value=1.0, description='ばね:', max=10.0, step=0.1)

BoundedIntText(value=30, description='ホライズン長:', max=1000, min=1)

BoundedFloatText(value=0.01, description='ステップ時間:', min=1e-05)

interactive(children=(FloatSlider(value=10.0, description='Qxx', layout=Layout(width='100%'), style=SliderStyl…